In [391]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
from tsetlin import TsetlinMachine
import torch
import random

DATASET_DIR = '../datasets/'
DATA_FILE = 'bit_1.txt'

text_rows = open(f'{DATASET_DIR}{DATA_FILE}', 'r').read().splitlines()
dataset = [ [int(num) for num in row.split(',')] for row in text_rows]
tensor_dataset = torch.tensor(dataset)
train_x = tensor_dataset[:, :-1]
train_y = tensor_dataset[:, -1]

# seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
# random.seed(seed)
# torch.manual_seed(seed)
# print(seed)
# random.seed(1320387042447901345)
# torch.manual_seed(1320387042447901345)
tm = TsetlinMachine(train_x.shape[1], 5)
for i in range(6):
    shuffled_idx = torch.randperm(train_x.shape[0])
    shuffled_x = train_x[shuffled_idx]
    shuffled_y = train_y[shuffled_idx]
    for x,y in zip(shuffled_x, shuffled_y):
        out_1 = tm.forward(x.unsqueeze(0))
        tm.update(y.unsqueeze(0))
        out_2 = tm.forward(x.unsqueeze(0))
        assert torch.equal(y.unsqueeze(0),out_2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [392]:
tm.l1.W,tm.l2.W[:, :tm.l2.W.shape[1]//2],tm.l2.W[:, tm.l2.W.shape[1]//2:]

(tensor([[ 0,  0, 37, 37, 37,  0],
         [ 0, 37, 37, 32,  0,  0],
         [ 0,  0,  0, 16,  0, 16],
         [37,  0,  0,  0, 32,  0],
         [ 0,  0,  0,  0,  0, 22]]),
 tensor([[0, 0, 0, 0, 0]]),
 tensor([[31, 31, 15,  0,  0]]))

In [393]:
out = tm.forward(train_x)
assert torch.equal(train_y, out)

In [394]:
failed_count = 0
for _ in range(1000):
    tm = TsetlinMachine(train_x.shape[1], 5)
    for i in range(6):
        shuffled_idx = torch.randperm(train_x.shape[0])
        shuffled_x = train_x[shuffled_idx]
        shuffled_y = train_y[shuffled_idx]

        for j, (x, y) in enumerate(zip(shuffled_x, shuffled_y)):
            out_1 = tm.forward(x.unsqueeze(0))
            tm.update(y.unsqueeze(0))
            out_2 = tm.forward(x.unsqueeze(0))
            assert torch.equal(y.unsqueeze(0), out_2)

    out = tm.forward(train_x)
    if not torch.equal(train_y, out):
        failed_count += 1

failed_count

187

In [459]:
tm = TsetlinMachine(train_x.shape[1], 5)
out_1 = tm.forward(train_x)
out_1, train_y

(tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([1, 1, 1, 1, 0, 0, 0, 0]))

In [460]:
Y = torch.randint(0,2, tm.l1.out.size())
tm.l1.out, Y, tm.l1.W

(tensor([[0, 0, 1, 0, 0],
         [0, 1, 1, 1, 1],
         [0, 0, 1, 0, 0],
         [1, 1, 1, 1, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]),
 tensor([[1, 0, 0, 0, 1],
         [1, 1, 0, 0, 0],
         [1, 1, 0, 0, 1],
         [1, 0, 1, 1, 0],
         [1, 0, 1, 0, 0],
         [1, 0, 0, 0, 1],
         [0, 1, 1, 0, 0],
         [1, 0, 1, 1, 0]]),
 tensor([[1, 1, 1, 0, 0, 0],
         [1, 0, 1, 0, 0, 0],
         [1, 0, 0, 0, 0, 0],
         [1, 0, 1, 0, 0, 0],
         [1, 0, 1, 0, 1, 0]]))

In [461]:
flip_deps = []
for i, single_x in enumerate(tm.l1.full_X):
    flip_dep_row = [[] for _ in range(tm.l1.in_dim * 2)]
    single_Y = Y[i]
    if not torch.equal(single_Y, tm.l1.out[i]):
        one_Y_idxs = torch.nonzero(single_Y == 1).squeeze(1)
        W_halves = torch.split(tm.l1.W, tm.l1.in_dim, dim=1)
        pos_W = W_halves[0]
        neg_W = W_halves[1]
        for pos_one_Y_idx in one_Y_idxs:
            w_1 = pos_W[pos_one_Y_idx]
            for neg_one_Y_idx in one_Y_idxs:
                w_2 = neg_W[neg_one_Y_idx]
                deps = ((w_1 == w_2) & (w_1 == 1))
                if deps.any():
                    dep_idxs = deps.nonzero(as_tuple=True)[0]
                    for idx in dep_idxs:
                        if pos_one_Y_idx.item() not in flip_dep_row[idx]:
                            flip_dep_row[idx].append(pos_one_Y_idx.item())
                        if neg_one_Y_idx.item() not in flip_dep_row[idx + tm.l1.in_dim]:
                            flip_dep_row[idx + tm.l1.in_dim].append(neg_one_Y_idx.item())
    flip_deps.append(flip_dep_row)
flip_deps

[[[], [0], [], [], [4], []],
 [[], [], [], [], [], []],
 [[], [0], [], [], [4], []],
 [[], [], [], [], [], []],
 [[], [], [], [], [], []],
 [[], [0], [], [], [4], []],
 [[], [], [], [], [], []],
 [[], [], [], [], [], []]]

In [462]:
tm.l1.full_X

tensor([[1, 0, 0, 0, 1, 1],
        [1, 0, 1, 0, 1, 0],
        [1, 1, 0, 0, 0, 1],
        [1, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 1],
        [0, 1, 0, 1, 0, 1],
        [0, 0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0, 0]])

In [463]:
W_deps = []
for i, single_W in enumerate(tm.l1.W):
    single_Y = Y[:, i]
    single_out = tm.l1.out[:, i]
    one_Y_idxs = torch.nonzero(single_Y == 1).squeeze(1)
    x_prod = tm.l1.full_X[one_Y_idxs].prod(dim=0)    
    must_be_one_idxs = torch.nonzero(x_prod == 1).squeeze(1)
    zero_Y_idxs = torch.nonzero(single_Y == 0).squeeze(1)

    must_be_flipped_row = [[] for _ in range(tm.l1.in_dim * 2)]
    if len(must_be_one_idxs) > 0:
        x_stuff = tm.l1.full_X[zero_Y_idxs][:, must_be_one_idxs] == 1
        for row in range(x_stuff.shape[0]):
            for col in range(x_stuff.shape[1]):
                if x_stuff[row, col] and zero_Y_idxs[row].item() not in must_be_flipped_row[must_be_one_idxs[col]]:
                    must_be_flipped_row[must_be_one_idxs[col]].append(zero_Y_idxs[row].item())

    W_deps.append([must_be_one_idxs.tolist(), must_be_flipped_row])

W_deps

[[[], [[], [], [], [], [], []]],
 [[], [[], [], [], [], [], []]],
 [[], [[], [], [], [], [], []]],
 [[1, 2], [[], [2, 5], [1, 6], [], [], []]],
 [[5], [[], [], [], [], [], [4]]]]